# libs

In [91]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
import textwrap

# data uploading

In [107]:
# https://drive.google.com/drive/u/0/folders/1K5qV_VEYkVVA0gGtxO3eArKzbPhCEqnA
data_train = pd.read_csv('salary-train.csv') 
data_test = pd.read_csv('salary-test-mini.csv')

In [5]:
# enc = DictVectorizer()
# X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
# X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
# data_train['LocationNormalized'].fillna('nan', inplace=True)
# data_train['ContractTime'].fillna('nan', inplace=True)

In [11]:
# 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv 
# (либо его заархивированную версию salary-train.zip).

# 2. Проведите предобработку:
        # - Приведите тексты к нижнему регистру (text.lower()).
        # - Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. 
        # - Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). 
        # Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:
        # train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
        # - Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, 
        # которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
        # - Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. 
        # - Код для этого был приведен выше.
        # - Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
        # Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для 
        # текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться 
        # функцией scipy.sparse.hstack.

# 3.  Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. 
# Целевая переменная записана в столбце SalaryNormalized.

# 4.  Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются 
# ответом на задание. Укажите их через пробел.

In [108]:
data_train.columns = data_train.columns.str.lower()
data_train.fulldescription = data_train.fulldescription.str.lower()
data_train.locationnormalized = data_train.locationnormalized.str.lower()
data_train['fulldescription'] = data_train['fulldescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_train['locationnormalized'] = data_train['locationnormalized'].fillna('lol')
data_train['contracttime'] = data_train['contracttime'].fillna('lol')

In [109]:
data_test.columns = data_test.columns.str.lower()
data_test.fulldescription = data_test.fulldescription.str.lower()
data_test.locationnormalized = data_test.locationnormalized.str.lower()
data_test['fulldescription'] = data_test['fulldescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['locationnormalized'] = data_test['locationnormalized'].fillna('lol')
data_test['contracttime'] = data_test['contracttime'].fillna('lol')

In [44]:
# textwrap.wrap(data_train.fulldescription.iloc[0])

In [119]:
vectorizer = TfidfVectorizer(min_df=5)
enc = DictVectorizer()
X = vectorizer.fit_transform(data_train['fulldescription'])
X_categ = enc.fit_transform(data_train[['locationnormalized', 'contracttime']].to_dict('records'))
X_final = hstack((X, X_categ))

In [120]:
X_test = vectorizer.transform(data_test['fulldescription'])
X_test_categ = enc.transform(data_test[['locationnormalized', 'contracttime']].to_dict('records'))
X_final_test = hstack((X_test, X_test_categ))

In [121]:
y = data_train.salarynormalized.values

# hw1

In [125]:
ridge_model = Ridge(alpha=1, random_state=241)
ridge_model.fit(X_final, y)

Ridge(alpha=1, random_state=241)

In [126]:
predictions = ridge_model.predict(X_final_test).round(2)
predictions

array([56574.11, 37198.85])

# hw2

In [ ]:
# Загрузите данные close_prices.csv. В этом файле приведены цены акций 30 компаний 
# на закрытии торгов за каждый день периода. 

# На загруженных данных обучите преобразование PCA с числом компоненты равным 10. 
# Скольких компонент хватит, чтобы объяснить 90% дисперсии?

# Примените построенное преобразование к исходным данным и возьмите значения первой компоненты. 

# Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. Чему равна корреляция Пирсона 
# между первой компонентой и индексом Доу-Джонса? 

# Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.

# Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. 
# При необходимости округляйте дробную часть до двух знаков.

In [92]:
price = pd.read_csv('close_prices.csv')
index = pd.read_csv('djia_index.csv')
price.iloc[:,1:].head(3)

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999


In [93]:
pca = PCA(n_components=10)
price_pca = pca.fit_transform(price.iloc[:, 1:])
sum(pca.explained_variance_ratio_).round(2)

0.99

In [94]:
pd.DataFrame(pca.components_, columns=price.columns[1:]).head(3)

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,0.016138,0.120645,-0.051661,0.050484,-0.125860,0.114090,0.233906,-0.006205,0.251227,0.288996,...,0.023092,0.077732,-0.007206,0.189480,0.321564,0.053683,0.579684,0.000109,0.087161,-0.042942
1,-0.233026,0.138102,-0.564736,-0.029421,-0.297156,-0.070072,-0.241633,-0.022603,-0.105527,0.031603,...,0.040701,-0.018652,-0.035559,-0.076710,-0.054117,-0.001830,0.106388,-0.032192,0.050303,-0.257983
2,-0.105902,-0.473845,-0.053749,0.007206,-0.048401,-0.066652,-0.045366,-0.031786,0.348309,0.142316,...,-0.070711,0.135307,-0.001104,0.091877,0.072012,-0.433768,-0.375350,0.022252,-0.012916,-0.164835


In [98]:
pca_test = PCA(n_components=3)
price_pca_test = pca_test.fit_transform(price.iloc[:, 1:])
print(sum(pca_test.explained_variance_ratio_).round(2))
pd.DataFrame(pca_test.components_, columns=price.columns[1:])

0.9


,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,0.016138,0.120645,-0.051661,0.050484,-0.125860,0.114090,0.233906,-0.006205,0.251227,0.288996,...,0.023092,0.077732,-0.007206,0.189480,0.321564,0.053683,0.579684,0.000109,0.087161,-0.042942
1,-0.233026,0.138102,-0.564736,-0.029421,-0.297156,-0.070072,-0.241633,-0.022603,-0.105527,0.031603,...,0.040701,-0.018652,-0.035559,-0.076710,-0.054117,-0.001830,0.106388,-0.032192,0.050303,-0.257983
2,-0.105902,-0.473845,-0.053749,0.007206,-0.048401,-0.066652,-0.045366,-0.031786,0.348309,0.142316,...,-0.070711,0.135307,-0.001104,0.091877,0.072012,-0.433768,-0.375350,0.022252,-0.012916,-0.164835


In [96]:
pd.DataFrame(pca.components_, columns=price.columns[1:]).iloc[0].sort_values(ascending=False).head(1)

V    0.579684
Name: 0, dtype: float64

In [97]:
result_pca = pd.DataFrame(price_pca, columns=[  'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 
                                                'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
X = result_pca['PC1'].values
y = index['^DJI'].values
np.corrcoef(X, y).round(2)

array([[1.  , 0.91],
       [0.91, 1.  ]])